In [1]:
import sys
import os
import numpy as np
from matplotlib import pyplot as plt
import matplotlib as mpl
import matplotlib.patches as patches
from shapely.geometry import Point
from shapely.geometry import MultiPoint
from shapely.geometry import MultiPolygon
from shapely.geometry import Polygon
from shapely.geometry import box
from descartes.patch import PolygonPatch
import ipywidgets as widgets
import ipywidgets.widgets.interaction
#from ipywidgets import interactive, fixed
from ipywidgets import *
import random
import pandas as pd

import re

from shapely import geometry
import pickle
import time

from ipyparallel import Client

from shapely import affinity
from shapely.geometry import LineString
import warnings
warnings.simplefilter('ignore', np.RankWarning)
import gc
gc.enable()


In [2]:
#os.chdir('/home/charly/Lab/Projects/uJ/local/experiments/HT-LB/')



In [3]:
pwd()

'/home/esb/Projects/uJ/local/experiments/HT-Sine2hr'

In [4]:
#sys.path.insert(0, '/Users/ESB/RPM_SYNC/RPM_Work/uJ_master/current/uJ_src/python/')

#sys.path.insert(0, '/Users/ESB/SYNC_RPM/RPM_Work/uJ_master/current/uJ_src/python/')
#sys.path.insert(0, '/home/charly/Lab/Projects/uJ/uJ_master/current/uJ_src/python/')
sys.path.insert(0, '/home/charly/Lab/Projects/uJ/local/current/uJ_src_pHT/python/')
sys.path.insert(0, '/home/esb/Projects/uJ/local/current/uJ_src_pHT/python/')



In [5]:
from readroi import *
from DataManagers import *
from TrackingFunctions import *
from PlottingFunctions import *

Importing readroi
DataStructs...loaded!
DataManagers...loaded!
TrackingFunctions... loaded!
PlottingFunctions... loaded!


In [ ]:
# import sys, importlib
# importlib.reload(sys.modules['DataManagers'])
# importlib.reload(sys.modules['TrackingFunctions'])
# importlib.reload(sys.modules['PlottingFunctions'])
# from DataManagers import *
# from TrackingFunctions import *
# from PlottingFunctions import *

## Note 
### Start a cluster with ipcluster start

## Experiment Parameters

In [16]:

expeLabel="HT-Sine-2hr"
rootDir="../../data_sample/HT-Sine-2hr/"
trap_list="xy01,xy02,xy03,xy05,xy06,xy07,xy08,xy09,xy10,xy11,xy12,xy13,xy14,xy15,xy16,xy17,xy18,xy19,xy20,xy21,xy22,xy23,xy24,xy25,xy26,xy27,xy28,xy29,xy30,xy31,xy32".split(",")  
trap_list="xy19,xy19-1".split(",")  



#data_type="relativeIntensity"
#data_type="normData"
data_type="normData"


figxlim=640
figylim=512

flims=(figxlim,figylim)


#Input files
for trap in trap_list:
    #Input files
    dirNameROIs=rootDir+"data_rois/"+trap+"/DsRed+GFP/"
    #dirNameDATA=rootDir+'data/'+data_type+'/'+trap+"/"
    dirNameDATA=rootDir+'data/'+data_type+'/'+trap+"/"
    
    #Output
    dirNameCELLS=rootDir+'data_cells/'+trap+'/'+data_type+'/'
    if not os.path.exists(dirNameCELLS):
        print("making dir cells...")
        os.makedirs(dirNameCELLS)

    dirNameTRACKEDCELLS=rootDir+'data_cells_tracked/'+trap+'/'+data_type+'/'
    if not os.path.exists(dirNameTRACKEDCELLS):
        print("making dir tracked cells...")
        os.makedirs(dirNameTRACKEDCELLS)



    dirNameTRACKEDCELLS_IMGS=rootDir+'images_cells_tracked/'+trap+'/'+data_type+'/'
    if not os.path.exists(dirNameTRACKEDCELLS_IMGS):
        print("making dir tracked cells imgs ...")
        os.makedirs(dirNameTRACKEDCELLS_IMGS)

    dirNameDATALINEAGES=rootDir+'data/'
    if not os.path.exists(dirNameDATALINEAGES):
        print("making dir lineages...")
        os.makedirs(dirNameDATALINEAGES)
        
    dirNameFIGURES=rootDir+'figures/traked_cells/'+trap+'/'+data_type+'/'
    if not os.path.exists(dirNameFIGURES):
        print("making dir figures...")
        os.makedirs(dirNameFIGURES)
#toFile=False
toFile=True


frame_experiment_start=1;
frame_signal_start=int(24)
frame_signal_end=int(210)
frame_experiment_end=int(210)



frame_experiment_start=1;
frame_signal_start=24
frame_signal_end=116
frame_experiment_end=116



frames=range(int(frame_experiment_start),int(frame_experiment_end)+1)
frames_list=[x for x in frames]

making dir cells...
making dir tracked cells...
making dir tracked cells imgs ...
making dir figures...


___
# Create Cells


### Load ROIs generated by uJ

In [7]:

fileROIs=load_rois(dirNameROIs)
dataFiles=load_data_files(dirNameDATA)
numFrames=len(fileROIs)
#print('Loading %s frames'%numFrames)
#meanGFP-750print('Loading %s data files'%len(dataFiles))

fileROIs=fileROIs[0:frame_experiment_end]
dataFiles=dataFiles[0:frame_experiment_end]
#fileROIs=fileROIs[0:5]
#dataFiles=dataFiles[0:5]

numFrames=len(fileROIs)
print('Loading  %s frames'%numFrames)
print('Loading %s data files'%len(dataFiles))

    
   ##### NOTE if path not found check the NAME of your local sync folder 

Loading  116 frames
Loading 116 data files


### Process ROIs (parallel version)

In [8]:
def par_process_rois():
    
    import numpy as np
    
    def get_max_x_angle(poly):
        rangle=0
        angles= list(range(0,185,5))
        (bminx, bminy, bmaxx, bmaxy)=poly.bounds
        xl=bmaxx-bminx
        yl=bmaxy-bminy
        center=Point((xl/2)+bminx,(yl/2)+bminy)

        max_xrange=xl
        for i,angle in enumerate(angles):
            #print(i,angle)
            poly_t=affinity.rotate(poly,angle,center)
            (bminx, bminy, bmaxx, bmaxy)=poly_t.bounds
            xl=bmaxx-bminx

            if(xl>max_xrange):
                max_xrange=xl
                rangle=angle

        return rangle,center


    def get_cell_axis(poly):
      #  t=time.time()
        line=LineString()
        pre_cloud=[]
        #poly=poly.convex_hull
        angle,center=get_max_x_angle(poly)

        poly=affinity.rotate(poly, angle,center )

        ext=poly.exterior.coords

        decs=0

        for i,seed_point in enumerate(ext):
            pre_cloud_i=[]

            seed_point=Point(seed_point)

            i=.5;
            intersection=poly.exterior.intersection(seed_point)
            sphere=seed_point.buffer(i)
            intersection=poly.exterior.intersection(sphere.exterior)
            while(len(list(intersection))>0):
                arc=Point()
                sphere=seed_point.buffer(i)
                intersection=poly.exterior.intersection(sphere.exterior)
                i+=3
                #i+=.5

                if(len(list(intersection))<=1):
                    continue

                arc=poly.intersection(sphere.exterior)

                if(arc.geom_type=="MultiLineString"):

                    for this_geom in arc.geoms:
                        arc_xy=list(this_geom.coords)

                        for pi in arc_xy:
                            pi=Point(pi)
                            pre_cloud_i.append((round(pi.x,decs),round(pi.y,decs)))

                elif(arc.geom_type=="LineString"):
                    for pi in list(arc.coords):
                        pi=Point(pi) 
                        pre_cloud_i.append((round(pi.x,decs),round(pi.y,decs)))


            pre_cloud.append(pre_cloud_i)


        cloud=MultiPoint(pre_cloud[0])

        for line in pre_cloud:

            line=MultiPoint(line)
            cloud=cloud.union(line)

        ###make poly fit to points cloud
        xs=[]
        ys=[]
        for pi in cloud:
            pi=Point(pi)

            xs.append(pi.x)
            ys.append(pi.y)

        pf=np.poly1d(np.polyfit(xs,ys,25))
        xr=np.linspace(np.min(xs),np.max(xs),100)

        line=[]
        for x in xr:
            pi=Point(x,pf(x))
            line.append(pi)

        line=LineString(line)
        linexy=np.array(line)
        linex=linexy[:,0]
        liney=linexy[:,1]

        line=poly.intersection(line)
        liner=affinity.rotate(line,-angle,center)
        if(liner.geom_type=="MultiLineString"):
            line=LineString()
            for this_geom in liner.geoms:
                if(this_geom.length>line.length):
                    line=this_geom
            liner=line



        linexy=np.array(liner)
        linex=linexy[:,0]
        liney=linexy[:,1]

        x=(np.max(xs)-np.min(xs))/2+np.min(xs)
        pi=Point(x,pf(x))


        center=affinity.rotate(pi,-angle,center)
       # t=time.time()-t
       # print(t)

        return liner,center
    
    frame_axis=[]
    frame_centers=[]
    for j,this_roi in enumerate(rois):
        
        this_axis,this_center=get_cell_axis(this_roi)
            
        frame_axis.append(this_axis)
        frame_centers.append(this_center)
    ret=[rois, roiIDs, frame_axis, frame_centers]
    return ret



In [11]:
re_do_cells=False
re_do_cells=True

trap_cells=[]
#trap_list="xy19".split(",")  



for trap in trap_list:
    dirNameROIs=rootDir+"data_rois/"+trap+"/DsRed+GFP/"
    dirNameDATA=rootDir+'data/'+data_type+'/'+trap+"/"
    dirNameCELLS=rootDir+'data_cells/'+trap+'/'+data_type+'/'
    fileROIs=load_rois(dirNameROIs)
    dataFiles=load_data_files(dirNameDATA)
    
    

    fileROIs=fileROIs[0:frame_experiment_end-frame_experiment_start+1]
    dataFiles=dataFiles[0:frame_experiment_end-frame_experiment_start+1]

    print('Loading %s frames'%len(fileROIs))
    print('Loading %s data files'%len(dataFiles))
#     fx=frame_experiment_end-frame_experiment_start
#     fileROIs=fileROIs[fx:frame_experiment_end-frame_experiment_start+1]
#     dataFiles=dataFiles[fx:frame_experiment_end-frame_experiment_start+1]



#     fileROIs=fileROIs[0:2]
#     dataFiles=dataFiles[0:2]

#     numFrames=len(fileROIs)
#     print('Loading  %s frames'%numFrames)
#     print('Loading %s data files'%len(dataFiles))



    if re_do_cells:
        rc = Client()
        dview = rc[:]
        with rc[:].sync_imports(): 
            from shapely.geometry import Point
            from shapely.geometry import LineString
            from shapely import affinity
            from shapely.geometry import MultiPoint


        global cells  # global?
        cells=[]
        for frame, this_file in enumerate(fileROIs):
            [rois,roiIDs]=load_frame('%s%s'%(dirNameROIs,this_file))
            this_data_file="%s%s"%(dirNameDATA,dataFiles[frame])
            
            #rois=[poly.convex_hull for poly in rois]

            print("\n(Parallel) Processing %s ROIs from %s"%(len(rois),this_file))


            tic()

            #First scatter rois
            dview.scatter('rois', rois)
            dview.scatter('roiIDs', roiIDs)

            #Asynchronous execution of par_process_rois
            ret=dview.apply_async(par_process_rois)

            #Now gather results
            frame_rois = []
            frame_roiIDs = []
            frame_axis = []
            frame_center = []

            ret_proc=ret.get()
            for i in rc.ids:
                frame_rois.append(ret_proc[i][0])
                frame_roiIDs.append(ret_proc[i][1])
                frame_axis.append(ret_proc[i][2])
                frame_center.append(ret_proc[i][3])

            frame_rois=[y for x in frame_rois for y in x]
            frame_roiIDs=[y for x in frame_roiIDs for y in x]
            frame_axis=[y for x in frame_axis for y in x]
            frame_center=[y for x in frame_center for y in x]

            toc()

            print("Compiling fluorescence from %s"%(dataFiles[frame]))
            this_cells=create_cells(frame,this_data_file, frame_roiIDs, frame_rois, frame_axis, frame_center,frame_experiment_start)
            cells.append(this_cells)

            fileNameCells="%s%s.pkl"%(dirNameCELLS,os.path.splitext(os.path.basename(this_file))[0])    
            save_cells(this_cells, fileNameCells)
    
        trap_cells.append(cells)
    
    else:
        print()
        #max_frames=-1
        cells=load_cells(dirNameCELLS)
        trap_cells.append(cells)
        ##### NOTE   if not working check for changes in your folder names due to sync

Loading 116 frames
Loading 116 data files
importing Point from shapely.geometry on engine(s)
importing LineString from shapely.geometry on engine(s)
importing affinity from shapely on engine(s)
importing MultiPoint from shapely.geometry on engine(s)


/home/esb/.anaconda3/lib/python3.6/site-packages/ipyparallel/client/client.py:459: RuntimeWarning: 
            Controller appears to be listening on localhost, but not on this machine.
            If this is true, you should specify Client(...,sshserver='you@strix')
            or instruct your controller to listen on an external IP.
  RuntimeWarning)



(Parallel) Processing 570 ROIs from HT-Sine-2hr_xy19_001.zip
Elapsed time: 35.227591 seconds.
Compiling fluorescence from HT-Sine-2hr_xy19_001.txt
Saving 570 cells to ../../data_sample/HT-Sine-2hr/data_cells/xy19/normData/HT-Sine-2hr_xy19_001.pkl

(Parallel) Processing 566 ROIs from HT-Sine-2hr_xy19_002.zip
Elapsed time: 36.056159 seconds.
Compiling fluorescence from HT-Sine-2hr_xy19_002.txt
Saving 566 cells to ../../data_sample/HT-Sine-2hr/data_cells/xy19/normData/HT-Sine-2hr_xy19_002.pkl

(Parallel) Processing 562 ROIs from HT-Sine-2hr_xy19_003.zip
Elapsed time: 37.399029 seconds.
Compiling fluorescence from HT-Sine-2hr_xy19_003.txt
Saving 562 cells to ../../data_sample/HT-Sine-2hr/data_cells/xy19/normData/HT-Sine-2hr_xy19_003.pkl

(Parallel) Processing 572 ROIs from HT-Sine-2hr_xy19_004.zip
Elapsed time: 34.975170 seconds.
Compiling fluorescence from HT-Sine-2hr_xy19_004.txt
Saving 572 cells to ../../data_sample/HT-Sine-2hr/data_cells/xy19/normData/HT-Sine-2hr_xy19_004.pkl

(Parall

Elapsed time: 41.427773 seconds.
Compiling fluorescence from HT-Sine-2hr_xy19_034.txt
Saving 485 cells to ../../data_sample/HT-Sine-2hr/data_cells/xy19/normData/HT-Sine-2hr_xy19_034.pkl

(Parallel) Processing 505 ROIs from HT-Sine-2hr_xy19_035.zip
Elapsed time: 44.966505 seconds.
Compiling fluorescence from HT-Sine-2hr_xy19_035.txt
Saving 505 cells to ../../data_sample/HT-Sine-2hr/data_cells/xy19/normData/HT-Sine-2hr_xy19_035.pkl

(Parallel) Processing 535 ROIs from HT-Sine-2hr_xy19_036.zip
Elapsed time: 43.305842 seconds.
Compiling fluorescence from HT-Sine-2hr_xy19_036.txt
Saving 535 cells to ../../data_sample/HT-Sine-2hr/data_cells/xy19/normData/HT-Sine-2hr_xy19_036.pkl

(Parallel) Processing 502 ROIs from HT-Sine-2hr_xy19_037.zip
Elapsed time: 45.741476 seconds.
Compiling fluorescence from HT-Sine-2hr_xy19_037.txt
Saving 502 cells to ../../data_sample/HT-Sine-2hr/data_cells/xy19/normData/HT-Sine-2hr_xy19_037.pkl

(Parallel) Processing 504 ROIs from HT-Sine-2hr_xy19_038.zip
Elapsed 


(Parallel) Processing 311 ROIs from HT-Sine-2hr_xy19_068.zip
Elapsed time: 49.457243 seconds.
Compiling fluorescence from HT-Sine-2hr_xy19_068.txt
Saving 311 cells to ../../data_sample/HT-Sine-2hr/data_cells/xy19/normData/HT-Sine-2hr_xy19_068.pkl

(Parallel) Processing 300 ROIs from HT-Sine-2hr_xy19_069.zip
Elapsed time: 48.676832 seconds.
Compiling fluorescence from HT-Sine-2hr_xy19_069.txt
Saving 300 cells to ../../data_sample/HT-Sine-2hr/data_cells/xy19/normData/HT-Sine-2hr_xy19_069.pkl

(Parallel) Processing 303 ROIs from HT-Sine-2hr_xy19_070.zip
Elapsed time: 47.889127 seconds.
Compiling fluorescence from HT-Sine-2hr_xy19_070.txt
Saving 303 cells to ../../data_sample/HT-Sine-2hr/data_cells/xy19/normData/HT-Sine-2hr_xy19_070.pkl

(Parallel) Processing 308 ROIs from HT-Sine-2hr_xy19_071.zip
Elapsed time: 51.422541 seconds.
Compiling fluorescence from HT-Sine-2hr_xy19_071.txt
Saving 308 cells to ../../data_sample/HT-Sine-2hr/data_cells/xy19/normData/HT-Sine-2hr_xy19_071.pkl

(Parall

Elapsed time: 67.243521 seconds.
Compiling fluorescence from HT-Sine-2hr_xy19_101.txt
Saving 322 cells to ../../data_sample/HT-Sine-2hr/data_cells/xy19/normData/HT-Sine-2hr_xy19_101.pkl

(Parallel) Processing 336 ROIs from HT-Sine-2hr_xy19_102.zip
Elapsed time: 73.823958 seconds.
Compiling fluorescence from HT-Sine-2hr_xy19_102.txt
Saving 336 cells to ../../data_sample/HT-Sine-2hr/data_cells/xy19/normData/HT-Sine-2hr_xy19_102.pkl

(Parallel) Processing 327 ROIs from HT-Sine-2hr_xy19_103.zip
Elapsed time: 66.248645 seconds.
Compiling fluorescence from HT-Sine-2hr_xy19_103.txt
Saving 327 cells to ../../data_sample/HT-Sine-2hr/data_cells/xy19/normData/HT-Sine-2hr_xy19_103.pkl

(Parallel) Processing 330 ROIs from HT-Sine-2hr_xy19_104.zip
Elapsed time: 65.919269 seconds.
Compiling fluorescence from HT-Sine-2hr_xy19_104.txt
Saving 330 cells to ../../data_sample/HT-Sine-2hr/data_cells/xy19/normData/HT-Sine-2hr_xy19_104.pkl

(Parallel) Processing 334 ROIs from HT-Sine-2hr_xy19_105.zip
Elapsed 

In [ ]:
trap_cells[0][0][1]

## View cells

In [ ]:
%matplotlib inline 

cells_plots_t=[]

for tracked_frame in range(len(cells)):
    this_fig=draw_cell(cells,tracked_frame+1)
    cells_plots_t.append(this_fig)



In [ ]:
%matplotlib inline 

this_tracked_plots=cells_plots_t
interactive_plot_t=interactive(show_all,this_tracked_plots=fixed(this_tracked_plots),tracked_frame=widgets.IntSlider(min=1,max=len(this_tracked_plots),step=1,value=1))
interactive_plot_t    



___
# uJ_Tracker: Step 2

### Select cells to track

In [ ]:
#global weight_vec
#weight_vec=[-.25,-.25,1,5,-5]
#[distance,angle diff,coverage,axis-intersection,relInt Diff]


In [ ]:
#%%html
#<style>
#.output_wrapper button.btn.btn-default,
#.output_wrapper .ui-dialog-titlebar {
#  display: none;
#}
#</style>

In [ ]:
trap_list.index("xy02")

In [ ]:
%matplotlib ipympl
#%matplotlib widgets

 #if plot does not show,uncomment this line run, comment and re-run
def onclickSelectMulti(event):

       #global max_frames

       #First find if Point is inside a roi
    this_point = Point(event.xdata,event.ydata)
    #if(event.y)
    
    title=event.inaxes.get_title()
    tindex=trap_list.index(title)
    to_track_polys=trap_to_track_polys[tindex]
    ax=axs[tindex]
    for select_index, select_poly in enumerate(to_track_polys):
        in_roi=select_poly.contains(this_point)
        if in_roi:
            select_x,select_y = select_poly.exterior.xy     
            ax.plot(select_x, select_y, color='#6699cc', alpha=0.7, linewidth=2, solid_capstyle='round', zorder=3)
            ax.axis('off')
            trap_toTrack_roiIndex[tindex].append(select_index)
            trap_toTrack_roiFrame[tindex].append(to_track_frame)
            #trap_toTrack_trapIndex
            
            break



fig,axs=plt.subplots(len(trap_cells),1,figsize=(30,30))
trap_to_track_polys=[]

trap_toTrack_roiIndex=[]
trap_toTrack_roiFrame=[]

for ti,t_cells in enumerate(trap_cells):
    cells=t_cells
    to_track_frame=len(cells)-1
    #to_track_frame=14
   # global toTrack_roiIndex
    trap_toTrack_roiIndex.append([])
    trap_toTrack_roiFrame.append([])




    
    #fig=plt.figure()
    ax = axs[ti]

    ax.set_xlim(0,640)
    ax.set_ylim(0,512)
    #ax.clear()
    to_track_polys=[]

    for this_cell in cells[to_track_frame]:
        this_poly=this_cell['roiPoly']
        to_track_polys.append(this_poly)
        patch = PolygonPatch(this_poly, facecolor=[0.75,0.75,0.75], edgecolor=[0,0,0], alpha=0.7, zorder=2)
        ax.add_patch(patch)
       #ax.plot(x=100,y=100,c='r*')
    #ax.axis('off')
    trap_to_track_polys.append(to_track_polys)
    ax.set_title(trap_list[ti])
    ax.set_aspect('equal')
    #fig.canvas.layout.width = '1280px'
    fig.canvas.layout.height = '%spx'%(512*len(trap_cells))
    fig.canvas.layout.width = '640px'
    #fig.canvas.layout.height = '512px'
    
    


cid = fig.canvas.mpl_connect('button_press_event', onclickSelectMulti)
#cid = plt.gcf().canvas.mpl_connect('button_press_event', onclickSelect)
    
   

In [ ]:
#%matplotlib notebook
## #%matplotlib inline   #if plot does not show,uncomment this line run, comment and re-run

#global max_frames
#max_frames=len(cells)-1

#global toTrack_roiIndex
#toTrack_roiIndex=[]

#global toTrack_roiFrame
#toTrack_roiFrame=[]

#track_all_cells=True
#track_all_cells=False

#def onclickSelect(event):
    
#    #global max_frames
#    
#    #First find if Point is inside a roi
#    this_point = Point(event.xdata,event.ydata)
#    
#    for select_index, select_poly in enumerate(to_track_polys):
#        in_roi=select_poly.contains(this_point)
#        if in_roi:
#            
#            select_x,select_y = select_poly.exterior.xy     
#            ax.plot(select_x, select_y, color='#6699cc', alpha=0.7, linewidth=2, solid_capstyle='round', zorder=3)
#            ax.axis('off')
            
#            toTrack_roiIndex.append(select_index)
#            toTrack_roiFrame.append(max_frames)
#            
#            break

#fig=plt.figure(figsize=(15,15))
#ax = plt.axes()
#ax.set_aspect('equal')
#plt.xlim(0,640)
#plt.ylim(0,512)
##ax.clear()

#to_track_polys=[]

#for this_cell in cells[max_frames]:
#    this_poly=this_cell['roiPoly']
#    to_track_polys.append(this_poly)
#    patch = PolygonPatch(this_poly, facecolor=[0.75,0.75,0.75], edgecolor=[0,0,0], alpha=0.7, zorder=2)
#    ax.add_patch(patch)
#    #ax.plot(x=100,y=100,c='r*')

#if track_all_cells:    
#    for select_index, this_poly in enumerate(to_track_polys):
#        patch = PolygonPatch(this_poly, facecolor='#6699cc', alpha=0.7, linewidth=2,  zorder=3)
#        ax.add_patch(patch)
#        
#        toTrack_roiIndex.append(select_index)
#        toTrack_roiFrame.append(max_frames)
    
#    ax.axis('off')
#    
#else:
    
#    ax.axis('off')
#   # plt.close()
#    cid = fig.canvas.mpl_connect('button_press_event', onclickSelect)
#    


In [ ]:
print(trap_toTrack_roiIndex)
print(trap_toTrack_roiFrame)

### Track selected cells

In [ ]:
#print(toTrack_roiIndex)
#print(toTrack_roiFrame)
weight_vec=[-5,-100,200,1500,-5,-5]
#weight_vec=[-1,-5,5,35,-175]
#[distance,angle_diff,coverage,axisFraction,GFPDiff,DsRedDiff]
trap_tracked_cells=[]
for ti,t_cells in enumerate(trap_cells):
    cells=t_cells
    cells=clear_tracks(cells)
    
    cells_tracked=cells.copy()
    if('cells_tracked' in vars()):
        cells_tracked=clear_tracks(cells_tracked)
    toTrack_roiIndex=trap_toTrack_roiIndex[ti]
    toTrack_roiFrame=trap_toTrack_roiFrame[ti]
    
    cells_tracked=track_this_indexs(toTrack_roiIndex,toTrack_roiFrame,cells,weight_vec) 
    
    trap_tracked_cells.append(cells_tracked)
    print('\n\nTracking complete! Trap:',trap)
    
    print("\nSaving cells...")
    file_cells= list(f for f in os.listdir(dirNameCELLS) if f.endswith('.pkl'))
    file_cells.sort()
    for frame, this_file in enumerate(file_cells):
        this_cells=cells_tracked[frame] 
        #print(frame, this_file)
        fileNameTrackedCells="%s%s.pkl"%(dirNameTRACKEDCELLS,os.path.splitext(os.path.basename(this_file))[0])    
    #    print(fileNameFrameCells)
        save_cells(this_cells, fileNameTrackedCells)




## Track cells from last frame


In [ ]:
#trap_list2=[trap_list[0]]
trap_tracked_cells=[]
for tindex,trap in enumerate(trap_list):

    dirNameCELLS=rootDir+'data_cells/'+trap+'/'+data_type+'/'
    dirNameTRACKEDCELLS=rootDir+'data_cells_tracked/'+trap+'/'+data_type+'/'
    print("Tracking trap: ",trap)
    
    cells=trap_cells[tindex]
    
    cells=clear_tracks(cells)
    cells_tracked=cells.copy()
    

    
    max_frames=len(cells)-1
    #max_frames=len(cells)-1

    from_frame=max_frames
    to_frame=max_frames-1
    
    for this_frame in range(from_frame,to_frame,-1):
        toTrack_roiIndex=[]
        toTrack_roiFrame=[]
        for select_index, this_cell in enumerate(cells_tracked[this_frame]):
            if len(this_cell['trackID'])==0:
                toTrack_roiFrame.append(this_frame)
                toTrack_roiIndex.append(select_index)
        print('\n___\nTracking %s cells from frame %s'%(len(toTrack_roiFrame),this_frame))
        cells_tracked=track_this_indexs(toTrack_roiIndex,toTrack_roiFrame,cells_tracked,weight_vec) 
    print('\n\nTracking complete! Trap:',trap)
    trap_tracked_cells.append(cells_tracked)
    print("\nSaving cells...")
    file_cells= list(f for f in os.listdir(dirNameCELLS) if f.endswith('.pkl'))
    file_cells.sort()
    for frame, this_file in enumerate(file_cells):
        this_cells=cells_tracked[frame] 
        #print(frame, this_file)
        fileNameTrackedCells="%s%s.pkl"%(dirNameTRACKEDCELLS,os.path.splitext(os.path.basename(this_file))[0])    
    #    print(fileNameFrameCells)
        save_cells(this_cells, fileNameTrackedCells)

### Track all cells all frames

In [12]:
#weight_vec=[-5,-100,200,1500,-5,-5]

#weight_vec=[-5,-1,150,150,-5,-5]
weight_vec=[-5,-100,200,1500,-5,-5]

trap_tracked_cells=[]

for tindex,trap in enumerate(trap_list):
    dirNameCELLS=rootDir+'data_cells/'+trap+'/'+data_type+'/'
    dirNameTRACKEDCELLS=rootDir+'data_cells_tracked/'+trap+'/'+data_type+'/'
    print("Tracking trap: ",trap)
    
    cells=trap_cells[tindex]
    
    cells=clear_tracks(cells)
    cells_tracked=cells.copy()
    
    max_frames=len(cells)-1
    from_frame=max_frames
    to_frame=0
    
    for this_frame in range(from_frame,to_frame,-1):
        toTrack_roiIndex=[]
        toTrack_roiFrame=[]
        for select_index, this_cell in enumerate(cells_tracked[this_frame]):
            if len(this_cell['trackID'])==0:
                toTrack_roiFrame.append(this_frame)
                toTrack_roiIndex.append(select_index)
        print('\n___\nTracking %s cells from frame %s'%(len(toTrack_roiFrame),this_frame))
        cells_tracked=track_this_indexs(toTrack_roiIndex,toTrack_roiFrame,cells_tracked,weight_vec) 
    print('\n\nTracking complete! Trap:',trap)
    trap_tracked_cells.append(cells_tracked)
    print("\nSaving cells...")
    file_cells= list(f for f in os.listdir(dirNameCELLS) if f.endswith('.pkl'))
    file_cells.sort()
    for frame, this_file in enumerate(file_cells):
        this_cells=cells_tracked[frame] 
        #print(frame, this_file)
        fileNameTrackedCells="%s%s.pkl"%(dirNameTRACKEDCELLS,os.path.splitext(os.path.basename(this_file))[0])    
    #    print(fileNameFrameCells)
        save_cells(this_cells, fileNameTrackedCells)
    

Tracking trap:  xy19

___
Tracking 322 cells from frame 115

0....................................................................................................
100....................................................................................................
200....................................................................................................
300......................
___
Tracking 34 cells from frame 114

0..................................
___
Tracking 47 cells from frame 113

0...............................................
___
Tracking 31 cells from frame 112

0...............................
___
Tracking 34 cells from frame 111

0..................................
___
Tracking 26 cells from frame 110

0..........................
___
Tracking 33 cells from frame 109

0.................................
___
Tracking 26 cells from frame 108

0..........................
___
Tracking 39 cells from frame 107

0.......................................
___
Tracking 

0.........................................................
___
Tracking 55 cells from frame 15

0.......................................................
___
Tracking 40 cells from frame 14

0........................................
___
Tracking 63 cells from frame 13

0...............................................................
___
Tracking 60 cells from frame 12

0............................................................
___
Tracking 48 cells from frame 11

0................................................
___
Tracking 59 cells from frame 10

0...........................................................
___
Tracking 46 cells from frame 9

0..............................................
___
Tracking 63 cells from frame 8

0...............................................................
___
Tracking 66 cells from frame 7

0..................................................................
___
Tracking 56 cells from frame 6

0........................................................

Saving 308 cells to ../../data_sample/HT-Sine-2hr/data_cells_tracked/xy19/normData/HT-Sine-2hr_xy19_066.pkl
Saving 303 cells to ../../data_sample/HT-Sine-2hr/data_cells_tracked/xy19/normData/HT-Sine-2hr_xy19_067.pkl
Saving 311 cells to ../../data_sample/HT-Sine-2hr/data_cells_tracked/xy19/normData/HT-Sine-2hr_xy19_068.pkl
Saving 300 cells to ../../data_sample/HT-Sine-2hr/data_cells_tracked/xy19/normData/HT-Sine-2hr_xy19_069.pkl
Saving 303 cells to ../../data_sample/HT-Sine-2hr/data_cells_tracked/xy19/normData/HT-Sine-2hr_xy19_070.pkl
Saving 308 cells to ../../data_sample/HT-Sine-2hr/data_cells_tracked/xy19/normData/HT-Sine-2hr_xy19_071.pkl
Saving 309 cells to ../../data_sample/HT-Sine-2hr/data_cells_tracked/xy19/normData/HT-Sine-2hr_xy19_072.pkl
Saving 327 cells to ../../data_sample/HT-Sine-2hr/data_cells_tracked/xy19/normData/HT-Sine-2hr_xy19_073.pkl
Saving 311 cells to ../../data_sample/HT-Sine-2hr/data_cells_tracked/xy19/normData/HT-Sine-2hr_xy19_074.pkl
Saving 302 cells to ../../da

In [23]:
trap_list2=trap_list.copy()
trap_list2=["xy19-1"]
trap_cells2=trap_cells.copy()

#trap_tracked_cells=[]
for tindex,trap in enumerate(trap_list2):

    dirNameCELLS=rootDir+'data_cells/'+trap+'/'+data_type+'/'
    dirNameCELLS=rootDir+'data_cells/xy19/'+data_type+'/'
    dirNameTRACKEDCELLS=rootDir+'data_cells_tracked/'+trap+'/'+data_type+'/'
    print("Tracking trap: ",trap)
    
    cells=trap_cells2[tindex]
    
    cells=clear_tracks(cells)
    cells_tracked=cells.copy()
    

    
    max_frames=len(cells)-1
    #max_frames=len(cells)-1

    from_frame=max_frames
    to_frame=max_frames-1
    
    for this_frame in range(from_frame,to_frame,-1):
        toTrack_roiIndex=[]
        toTrack_roiFrame=[]
        for select_index, this_cell in enumerate(cells_tracked[this_frame]):
            if len(this_cell['trackID'])==0:
                toTrack_roiFrame.append(this_frame)
                toTrack_roiIndex.append(select_index)
        print('\n___\nTracking %s cells from frame %s'%(len(toTrack_roiFrame),this_frame))
        cells_tracked=track_this_indexs(toTrack_roiIndex,toTrack_roiFrame,cells_tracked,weight_vec) 
    print('\n\nTracking complete! Trap:',trap)
    trap_tracked_cells.append(cells_tracked)
    print("\nSaving cells...")
    file_cells= list(f for f in os.listdir(dirNameCELLS) if f.endswith('.pkl'))
    file_cells.sort()
    for frame, this_file in enumerate(file_cells):
        this_cells=cells_tracked[frame] 
        #print(frame, this_file)
        fileNameTrackedCells="%s%s.pkl"%(dirNameTRACKEDCELLS,os.path.splitext(os.path.basename(this_file))[0])    
    #    print(fileNameFrameCells)
        save_cells(this_cells, fileNameTrackedCells)


Tracking trap:  xy19-1

___
Tracking 322 cells from frame 115

0....................................................................................................
100....................................................................................................
200....................................................................................................
300......................

Tracking complete! Trap: xy19-1

Saving cells...
Saving 570 cells to ../../data_sample/HT-Sine-2hr/data_cells_tracked/xy19-1/normData/HT-Sine-2hr_xy19_001.pkl
Saving 566 cells to ../../data_sample/HT-Sine-2hr/data_cells_tracked/xy19-1/normData/HT-Sine-2hr_xy19_002.pkl
Saving 562 cells to ../../data_sample/HT-Sine-2hr/data_cells_tracked/xy19-1/normData/HT-Sine-2hr_xy19_003.pkl
Saving 572 cells to ../../data_sample/HT-Sine-2hr/data_cells_tracked/xy19-1/normData/HT-Sine-2hr_xy19_004.pkl
Saving 567 cells to ../../data_sample/HT-Sine-2hr/data_cells_tracked/xy19-1/normData/HT-Sine-2hr_xy19_005.pkl
S

Saving 327 cells to ../../data_sample/HT-Sine-2hr/data_cells_tracked/xy19-1/normData/HT-Sine-2hr_xy19_073.pkl
Saving 311 cells to ../../data_sample/HT-Sine-2hr/data_cells_tracked/xy19-1/normData/HT-Sine-2hr_xy19_074.pkl
Saving 302 cells to ../../data_sample/HT-Sine-2hr/data_cells_tracked/xy19-1/normData/HT-Sine-2hr_xy19_075.pkl
Saving 296 cells to ../../data_sample/HT-Sine-2hr/data_cells_tracked/xy19-1/normData/HT-Sine-2hr_xy19_076.pkl
Saving 292 cells to ../../data_sample/HT-Sine-2hr/data_cells_tracked/xy19-1/normData/HT-Sine-2hr_xy19_077.pkl
Saving 275 cells to ../../data_sample/HT-Sine-2hr/data_cells_tracked/xy19-1/normData/HT-Sine-2hr_xy19_078.pkl
Saving 282 cells to ../../data_sample/HT-Sine-2hr/data_cells_tracked/xy19-1/normData/HT-Sine-2hr_xy19_079.pkl
Saving 272 cells to ../../data_sample/HT-Sine-2hr/data_cells_tracked/xy19-1/normData/HT-Sine-2hr_xy19_080.pkl
Saving 274 cells to ../../data_sample/HT-Sine-2hr/data_cells_tracked/xy19-1/normData/HT-Sine-2hr_xy19_081.pkl
Saving 282

In [25]:
trap_list

['xy19', 'xy19-1']

In [ ]:
len(cells_tracked)

### Load tracked cells

In [26]:
trap_tracked_cells=[]
for trap in trap_list:
    
    dirNameTRACKEDCELLS=rootDir+'data_cells_tracked/'+trap+'/'+data_type+'/'
    print(dirNameTRACKEDCELLS)
    tracked_cells=load_cells(dirNameTRACKEDCELLS)
    trap_tracked_cells.append(tracked_cells)

../../data_sample/HT-Sine-2hr/data_cells_tracked/xy19/normData/
115: Loading HT-Sine-2hr_xy19_116.pkl
../../data_sample/HT-Sine-2hr/data_cells_tracked/xy19-1/normData/
115: Loading HT-Sine-2hr_xy19_116.pkl


### Draw ROIs + Tracked Cells

In [ ]:
# def draw_cell(local_cells,tracked_frame):
#     fig=plt.figure(figsize=(15,15))
#     ax = plt.axes()
#     ax.set_aspect('equal')
#     #plt.xlim(0,640)
#     #plt.ylim(0,512)
#     #plt.axis('off')
#     ax.clear()
#     print('Building plot for frame frame %s '%(tracked_frame),end="\r")
#     for this_cell in local_cells[tracked_frame-1]:
        
#         this_poly=this_cell['roiPoly']
#         if(this_cell['trackID']):
#             patch = PolygonPatch(this_poly, facecolor=this_cell['cellColor'], edgecolor=[0.2,0.2,0.2], alpha=0.7, zorder=3)    
#         else:
#             patch = PolygonPatch(this_poly, facecolor=[0.75,0.75,0.75], edgecolor=[0,0,0], alpha=0.3, zorder=2)
        
#         ax.add_patch(patch)
#         this_axis=this_cell['axis']
#         linexy=np.array(this_axis)
#         linex=linexy[:,0]
#         liney=linexy[:,1]
#         ax.plot(linex,liney,'c-',alpha=0.5)
        
#     ax.axis('off')
#     plt.xlim(0,640)
#     plt.ylim(0,512)
#     rfig=fig
#     plt.close()
#     return rfig


['xy19', 'xy19-1', 'xy19-1']

In [27]:
%matplotlib inline
#max_frames=len(cells_tracked)
trap_tracked_plots=[]
for tindex,trap in enumerate(trap_list):
    tracked_plots=[]
    cells_tracked=trap_tracked_cells[tindex]
    dirNameFIGURES=rootDir+'figures/traked_cells/'+trap+'/'+data_type+'/'
    print("Trap: %s in %s"%(trap,dirNameFIGURES))
    for i,tracked_frame in enumerate(frames_list):
        this_fig=draw_cell(cells_tracked,i+1,flims)
        #plt.show(this_fig)
        tracked_frame_name=str(tracked_frame).zfill(3)
        figname="%s%s_%s-%s.png"%(dirNameFIGURES,expeLabel,trap,tracked_frame_name)
        this_fig.savefig(figname)
        tracked_plots.append(this_fig)
       # display(this_fig)
        plt.close("all")
    print()
    trap_tracked_plots.append(tracked_plots)

Trap: xy19 in ../../data_sample/HT-Sine-2hr/figures/traked_cells/xy19/normData/
Building plot for frame frame 116 
Trap: xy19-1 in ../../data_sample/HT-Sine-2hr/figures/traked_cells/xy19-1/normData/
Building plot for frame frame 116 


In [28]:
this_tracked_plots=trap_tracked_plots[0]
#interactive_plot=interactive(show_all,this_tracked_plots=fixed(this_tracked_plots),tracked_frame=widgets.IntSlider(min=1,max=max_frames,step=1,value=1),fs=fixed(frame_experiment_start))
interactive_plot=interactive(show_all,this_tracked_plots=fixed(this_tracked_plots),tracked_frame=widgets.IntSlider(min=frame_experiment_start,max=frame_experiment_end,step=1,value=1),fs=fixed(frame_experiment_start))
interactive_plot    



interactive(children=(IntSlider(value=1, description='tracked_frame', max=116, min=1), Output()), _dom_classes…

In [29]:
this_tracked_plots=trap_tracked_plots[1]
#interactive_plot=interactive(show_all,this_tracked_plots=fixed(this_tracked_plots),tracked_frame=widgets.IntSlider(min=1,max=max_frames,step=1,value=1),fs=fixed(frame_experiment_start))
interactive_plot=interactive(show_all,this_tracked_plots=fixed(this_tracked_plots),tracked_frame=widgets.IntSlider(min=frame_experiment_start,max=frame_experiment_end,step=1,value=1),fs=fixed(frame_experiment_start))
interactive_plot    



interactive(children=(IntSlider(value=1, description='tracked_frame', max=116, min=1), Output()), _dom_classes…

In [ ]:
%matplotlib inline 
max_frames=len(cells_tracked)
tracked_plots=[]
for tracked_frame in range(1,max_frames+1):
    this_fig=draw_cell(cells_tracked,tracked_frame)
    tracked_plots.append(this_fig)


In [ ]:
%matplotlib inline 




this_tracked_plots=tracked_plots
interactive_plot=interactive(show_all,this_tracked_plots=fixed(this_tracked_plots),tracked_frame=widgets.IntSlider(min=1,max=max_frames,step=1,value=1))
interactive_plot    



In [ ]:
%%html
<style>
.output_wrapper button.btn.btn-default,
.output_wrapper .ui-dialog-titlebar {
 display: none;
}
</style>